In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.4f' % x)
import seaborn as sns
sns.set_context("paper", font_scale=1.3)
sns.set_style('white')
import warnings
warnings.filterwarnings('ignore')
from time import time
import matplotlib.ticker as tkr
from scipy import stats
from statsmodels.tsa.stattools import adfuller
from sklearn import preprocessing
from statsmodels.tsa.stattools import pacf
%matplotlib inline

import math
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
# What version of Python do you have?
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

Tensor Flow Version: 2.1.1-dlenv_tfe
Keras Version: 2.3.0-tf

Python 3.7.6 | packaged by conda-forge | (default, Jun  1 2020, 18:57:50) 
[GCC 7.5.0]
Pandas 1.0.5
Scikit-Learn 0.23.1
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available


In [3]:
# from google.cloud import bigquery
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import glob


# def make_query_utility(fuse_project_id, fuse_dataset_id, fuse_bucket, fuse_bq_key):
    
#     BQ_CLIENT = bigquery.Client.from_service_account_json(fuse_bq_key)


#     def query(query_string):
#         formatted_string = query_string.format(project=fuse_project_id, dataset=fuse_dataset_id)
#         query_job = BQ_CLIENT.query(formatted_string)
#         dataframe = query_job.result().to_dataframe().astype(float, errors='ignore')
#         return dataframe

#     return query

# query = make_query_utility('kohls-merch-in-lle', 'FUSE_STG01', 'kohls-merch-fuse-dataload', 'bigquery_key.json')


# data = query("""
#   select *
#       from `kohls-merch-in-lle`.FUSE_LOCAL_TEST.DEMAND_TRAINING_15_543_WALKIN
#         where beginning_of_week between '2020-06-29' and  current_date()
        
       

# """)

In [4]:
# data.shape

In [5]:
# del data['total_revenue'], data['advertised_purchases']

In [6]:
# data.is_holiday = data.is_holiday.astype(int)
# data.node = data.node.astype(int)
# data.dept_nbr = data.dept_nbr.astype(int)

# data.cl_nbr = data.cl_nbr.astype(int)
# data.sub_cl_nbr = data.sub_cl_nbr.astype(int)


In [7]:
# data.info()

In [8]:
# data.to_csv('data_test_prepared_level1.csv',index=False)
data =pd.read_csv('data_test_prepared_level1.csv')
data.shape

(144156, 8)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144156 entries, 0 to 144155
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   total_sales        144156 non-null  float64
 1   is_holiday         144156 non-null  int64  
 2   node               144156 non-null  int64  
 3   dept_nbr           144156 non-null  int64  
 4   cl_nbr             144156 non-null  int64  
 5   sub_cl_nbr         144156 non-null  int64  
 6   subclass_id        144156 non-null  object 
 7   beginning_of_week  144156 non-null  object 
dtypes: float64(1), int64(5), object(2)
memory usage: 8.8+ MB


In [10]:
print("Min data from dataset:" , data['beginning_of_week'].min())
print("MAX data from dataset:" , data['beginning_of_week'].max())

Min data from dataset: 2020-07-05
MAX data from dataset: 2020-07-05


In [11]:
col = [ 'beginning_of_week', 'total_revenue', 'advertised_purchases']
data_train= pd.read_csv('data_train_prepared_level1.csv'  , usecols =col)
data_train.shape

(15713004, 3)

In [12]:
import datetime
from datetime import date
start_day_last_year=datetime.datetime.now() - datetime.timedelta(days = 1*365)
start_day_last_year= start_day_last_year.strftime('%Y-%m-%d')
start_day_last_year

'2019-07-12'

In [13]:
mask = data_train['beginning_of_week'] < start_day_last_year 
data_train = data_train.loc[mask]
data_train.beginning_of_week.max() , data_train.beginning_of_week.min()

('2019-07-07', '2018-06-03')

In [14]:
del data_train['beginning_of_week']
data_train= data_train.reset_index(drop=True)

In [15]:
data=pd.concat([data , data_train] ,axis=1)
data = data.dropna()

In [16]:
data.columns

Index(['total_sales', 'is_holiday', 'node', 'dept_nbr', 'cl_nbr', 'sub_cl_nbr',
       'subclass_id', 'beginning_of_week', 'total_revenue',
       'advertised_purchases'],
      dtype='object')

In [17]:
data['z_score_total_revenue']=stats.zscore(data['total_revenue'])
# data=data.loc[data['z_score_total_revenue'].abs()<=3]
data.shape 

(144156, 11)

In [18]:
data['z_score_advertised_purchases']=stats.zscore(data['advertised_purchases'])
# data=data.loc[data['z_score_total_sales'].abs()<=3]
data.shape

(144156, 12)

# Preprocssing Data

In [19]:
def is_holiday_func(col):
        data[col] = data[[col]].applymap(lambda x: 1 if x > 1 else 0)
        data[col].value_counts()

        
is_holiday_func('is_holiday')

In [20]:
import datetime
import pandas as pd
import numpy as np
data['beginning_of_week'] = pd.to_datetime( data['beginning_of_week'] , format='%Y-%m-%d')
def extract_date(col):
        # make the new columns using string indexing
        data['year'] = data[col].dt.year
        data['month'] = data[col].dt.month
        data['day'] = data[col].dt.day
        data['quarter']=pd.to_datetime(data[col]).dt.quarter
        data['smester']=np.where(data.quarter.isin([1,2]), 1 , 2)
        data['day_of_year']=pd.to_datetime(data[col]).dt.dayofyear
        data['week_of_year']=pd.to_datetime(data[col]).dt.weekofyear
        data['day_in_Month']=pd.to_datetime(data[col]).dt.days_in_month
        
extract_date('beginning_of_week')

In [21]:
from sklearn import preprocessing 
def encoding(col):
    label_encod = preprocessing.LabelEncoder()
    data[col] = label_encod.fit_transform(data[col])
    return data[col].value_counts()

# encoding('channel')
encoding('day_of_year')
encoding('year')

0    144156
Name: year, dtype: int64

In [22]:
data['subclass_id'] = data['subclass_id'].str.replace(r'-', '').astype(int)
data['subclass_id'].head()


0    151010
1    151010
2    151010
3    151010
4    151010
Name: subclass_id, dtype: int64

In [23]:
data.to_csv('data_test_prepared_level2.csv' , index=False)

In [24]:
data = pd.read_csv('data_test_prepared_level2.csv')

In [25]:
data.shape

(144156, 20)

In [26]:
print(data.columns)

Index(['total_sales', 'is_holiday', 'node', 'dept_nbr', 'cl_nbr', 'sub_cl_nbr',
       'subclass_id', 'beginning_of_week', 'total_revenue',
       'advertised_purchases', 'z_score_total_revenue',
       'z_score_advertised_purchases', 'year', 'month', 'day', 'quarter',
       'smester', 'day_of_year', 'week_of_year', 'day_in_Month'],
      dtype='object')


In [27]:
lag_list = [1, 2 , 3]


for lag in lag_list:
    ft_name = ('advertise_shifted%s' % lag)
    data[ft_name] = data['advertised_purchases'].shift(lag)
    data[ft_name].fillna(0, inplace=True)
    ft_name = ('total_revenue_shifted%s' % lag)
    data[ft_name] = data['total_revenue'].shift(lag)
    data[ft_name].fillna(0, inplace=True)



In [28]:
data.total_revenue_shifted1.max() , data.total_revenue_shifted1.min()


(3144.0, -312.07)

In [29]:
# Min value
f_min = lambda x: x.rolling(window=3, min_periods=1).min()
# Max value
f_max = lambda x: x.rolling(window=5, min_periods=1).max()
# Mean value
f_mean = lambda x: x.rolling(window=7, min_periods=1).mean()
# Standard deviation
f_std = lambda x: x.rolling(window=9, min_periods=1).std()

function_list = [f_min, f_max, f_mean]
function_name = ['min', 'max', 'mean']

for i in range(len(function_list)):
    data[('advertised_purchases_rolling_%s' % function_name[i])] = data.sort_values('beginning_of_week').groupby([ 'total_revenue'])['advertised_purchases'].apply(function_list[i])
    


In [30]:
# Min value
f_min = lambda x: x.rolling(window=3, min_periods=1).min()
# Max value
f_max = lambda x: x.rolling(window=5, min_periods=1).max()
# Mean value
f_mean = lambda x: x.rolling(window=7, min_periods=1).mean()
# Standard deviation
f_std = lambda x: x.rolling(window=9, min_periods=1).std()

function_list = [f_min, f_max, f_mean, f_std]
function_name = ['min', 'max', 'mean', 'std']

for i in range(len(function_list)):
    data[('total_revenue_rolling_%s' % function_name[i])] = data.sort_values('beginning_of_week').groupby([ 'advertised_purchases'])['total_revenue'].apply(function_list[i])
    data[('total_revenue_rolling_%s' % function_name[i])] .fillna(0 , inplace=True)

"Done"    

'Done'

In [31]:
data['expand_total_revenue_mean'] = data['total_revenue'].expanding(3).mean()
data['expand_total_revenue_mean'] .fillna(0, inplace=True)

data['expand_total_revenue_sum'] = data['total_revenue'].expanding(3).sum()
data['expand_total_revenue_sum'].fillna(0, inplace=True)

data['expand_advertised_purchases_mean'] = data['advertised_purchases'].expanding(3).mean()
data['expand_advertised_purchases_mean'].fillna(0, inplace=True)

data['expand_advertised_purchases_sum'] = data['advertised_purchases'].expanding(3).sum()
data['expand_advertised_purchases_sum'].fillna(0, inplace=True)


data['expand_node__sum'] =  data['node'].expanding(3).sum()
data['expand_node__sum'].fillna(0, inplace=True)
    
data['expand_node__mean'] =  data['node'].expanding(3).mean()
data['expand_node__mean'].fillna(0, inplace=True)

In [32]:
lag_list = [1, 2]


for lag in lag_list:
    ft_name = ('is_holiday_shifted%s' % lag)
    data[ft_name] = data['is_holiday'].shift(lag)
    data[ft_name].fillna(0, inplace=True)
    ft_name = ('node_shifted%s' % lag)
    data[ft_name] = data['node'].shift(lag)
    data[ft_name].fillna(0, inplace=True)


In [33]:
# Min value
f_min = lambda x: x.rolling(window=3, min_periods=1).min()
# Max value
f_max = lambda x: x.rolling(window=5, min_periods=1).max()
# Mean value
f_mean = lambda x: x.rolling(window=7, min_periods=1).mean()
# Standard deviation
f_std = lambda x: x.rolling(window=9, min_periods=1).std()

function_list = [f_min, f_max, f_mean]
function_name = ['min', 'max', 'mean']

for i in range(len(function_list)):
    data[('node_new_%s' % function_name[i])] = data.sort_values('beginning_of_week').groupby([ 'sub_cl_nbr'])['node'].apply(function_list[i])
    


data['expand_node__sum'] =  data['node'].expanding(3).sum()
data['expand_node__sum'].fillna(0, inplace=True)
    
data['expand_node__mean'] =  data['node'].expanding(3).mean()
data['expand_node__mean'].fillna(0, inplace=True)


In [34]:
from sklearn.preprocessing import MinMaxScaler
def MIN_MAX_SCALE(col):
    X = data[[col]].values # Modify to Numpy
    min_max_sc = preprocessing.MinMaxScaler()
    X_Scale = min_max_sc.fit_transform(X)
    data[col] = pd.DataFrame(X_Scale) # add to pandas DB


In [35]:
list_of_max_min = [ 'total_revenue', 'advertised_purchases', 'node', 
                    'advertise_shifted1', 'advertise_shifted2', 'advertise_shifted3', 'total_revenue_shifted1', 
                   'total_revenue_shifted2', 'total_revenue_shifted3', 'advertised_purchases_rolling_min', 
                   'advertised_purchases_rolling_max',
                   'advertised_purchases_rolling_mean', 'total_revenue_rolling_min', 'total_revenue_rolling_max',
                   'total_revenue_rolling_mean','total_revenue_rolling_std',
                    'expand_total_revenue_mean', 'expand_total_revenue_sum', 'expand_advertised_purchases_mean', 
                   'expand_advertised_purchases_sum',
                   'node_shifted1', 'node_shifted2', 'node_new_min', 'node_new_max', 'node_new_mean', 
                   'expand_node__sum', 'expand_node__mean']

for i in list_of_max_min:
    MIN_MAX_SCALE(i)



# print(len(list(data)))

In [36]:
print(data.columns )

Index(['total_sales', 'is_holiday', 'node', 'dept_nbr', 'cl_nbr', 'sub_cl_nbr',
       'subclass_id', 'beginning_of_week', 'total_revenue',
       'advertised_purchases', 'z_score_total_revenue',
       'z_score_advertised_purchases', 'year', 'month', 'day', 'quarter',
       'smester', 'day_of_year', 'week_of_year', 'day_in_Month',
       'advertise_shifted1', 'total_revenue_shifted1', 'advertise_shifted2',
       'total_revenue_shifted2', 'advertise_shifted3',
       'total_revenue_shifted3', 'advertised_purchases_rolling_min',
       'advertised_purchases_rolling_max', 'advertised_purchases_rolling_mean',
       'total_revenue_rolling_min', 'total_revenue_rolling_max',
       'total_revenue_rolling_mean', 'total_revenue_rolling_std',
       'expand_total_revenue_mean', 'expand_total_revenue_sum',
       'expand_advertised_purchases_mean', 'expand_advertised_purchases_sum',
       'expand_node__sum', 'expand_node__mean', 'is_holiday_shifted1',
       'node_shifted1', 'is_holiday_shif

In [42]:
col = ['total_sales', 'is_holiday', 'node', 'beginning_of_week', 'total_revenue',
       'advertised_purchases', 'z_score_total_revenue',
       'z_score_advertised_purchases', 'year', 'month', 'day', 'quarter',
       'smester', 'day_of_year', 'week_of_year', 'day_in_Month',
       'advertise_shifted1', 'total_revenue_shifted1', 'advertise_shifted2',
       'total_revenue_shifted2', 'advertise_shifted3',
       'total_revenue_shifted3', 'advertised_purchases_rolling_min',
       'advertised_purchases_rolling_max', 'advertised_purchases_rolling_mean',
       'total_revenue_rolling_min', 'total_revenue_rolling_max',
       'total_revenue_rolling_mean', 'total_revenue_rolling_std',
       'expand_total_revenue_mean', 'expand_total_revenue_sum',
       'expand_advertised_purchases_mean', 'expand_advertised_purchases_sum',
       'expand_node__sum', 'expand_node__mean', 'is_holiday_shifted1',
       'node_shifted1', 'is_holiday_shifted2', 'node_shifted2', 'node_new_min',
       'node_new_max', 'node_new_mean']

data = data.reindex(columns=col)


In [43]:
data.shape # 15476803

(144156, 42)

In [44]:
data.head().T

0           1           2  \
total_sales                            0.0000      0.0000      0.0000   
is_holiday                                  0           0           0   
node                                   0.1948      0.2030      0.1619   
beginning_of_week                  2020-07-05  2020-07-05  2020-07-05   
total_revenue                          0.0903      0.0903      0.0903   
advertised_purchases                   0.0000      0.0000      0.0000   
z_score_total_revenue                 -0.1394     -0.1394     -0.1394   
z_score_advertised_purchases          -0.1693     -0.1693     -0.1693   
year                                        0           0           0   
month                                       7           7           7   
day                                         5           5           5   
quarter                                     3           3           3   
smester                                     2           2           2   
day_of_year                                 0           0           0   
week_of_year                               27          27          27   
day_in_Month                               31          31          31   
advertise_shifted1                     0.0000      0.0000      0.0000   
total_revenue_shifted1                 0.0903      0.0903      0.0903   
advertise_shifted2                     0.0000      0.0000      0.0000   
total_revenue_shifted2                 0.0903      0.0903      0.0903   
advertise_shifted3                     0.0000      0.0000      0.0000   
total_revenue_shifted3                 0.0903      0.0903      0.0903   
advertised_purchases_rolling_min       0.0000      0.0000      0.0000   
advertised_purchases_rolling_max       0.0000      0.0000      0.0000   
advertised_purchases_rolling_mean      0.0000      0.0000      0.0000   
total_revenue_rolling_min              0.0903      0.0903      0.0903   
total_revenue_rolling_max              0.0089      0.0089      0.0089   
total_revenue_rolling_mean             0.0150      0.0150      0.0150   
total_revenue_rolling_std              0.0000      0.0000      0.0000   
expand_total_revenue_mean              0.0032      0.0032      0.0032   
expand_total_revenue_sum               0.0001      0.0001      0.0001   
expand_advertised_purchases_mean       0.0000      0.0000      0.0000   
expand_advertised_purchases_sum        0.0000      0.0000      0.0000   
expand_node__sum                       0.0000      0.0000      0.0000   
expand_node__mean                      0.0000      0.0000      0.9755   
is_holiday_shifted1                    0.0000      0.0000      0.0000   
node_shifted1                          0.0000      0.1956      0.2039   
is_holiday_shifted2                    0.0000      0.0000      0.0000   
node_shifted2                          0.0000      0.0000      0.1956   
node_new_min                           0.8314      0.0846      0.2295   
node_new_max                           0.1944      0.2027      0.2027   
node_new_mean                          0.6363      0.3665      0.4052   

                                            3           4  
total_sales                            0.0000      0.0000  
is_holiday                                  0           0  
node                                   0.1634      0.1825  
beginning_of_week                  2020-07-05  2020-07-05  
total_revenue                          0.0903      0.0903  
advertised_purchases                   0.0000      0.0000  
z_score_total_revenue                 -0.1394     -0.1394  
z_score_advertised_purchases          -0.1693     -0.1693  
year                                        0           0  
month                                       7           7  
day                                         5           5  
quarter                                     3           3  
smester                                     2           2  
day_of_year                                 0           0  
week

In [45]:
data.to_csv('data_test_prepared_level3.csv' , index=False)

In [41]:
print (" Done. ")

 Done. 
